<a href="https://colab.research.google.com/github/biniyam-mulugeta/test_co_2/blob/main/co_2_fe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
import numpy as np 
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Flatten,Conv2D
from keras.applications import densenet#mobilenet_v2#,efficientnet,vgg16,densenet
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from keras.utils.all_utils import to_categorical
from sklearn import preprocessing
from skimage.filters import threshold_otsu
import cv2
from pathlib import Path
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
%matplotlib inline

In [3]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [4]:
def k_means_segmentation(img):
  twoDimage = img.reshape((-1,3))
  twoDimage = np.float32(twoDimage)
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
  K = 2
  attempts=1
  ret,label,center=cv2.kmeans(twoDimage,K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
  center = np.uint8(center)
  res = center[label.flatten()]
  result_image = res.reshape((img.shape))
  return result_imag

def color_mask_segmentation(img):
  hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
  light_blue = (90, 70, 50)
  #dark_blue = (128, 255, 255)
  # You can use the following values for green
  light_green = (40, 40, 40)
  # dark_greek = (70, 255, 255)
  mask = cv2.inRange(hsv_img, light_blue, light_green)
  result = cv2.bitwise_and(img, img, mask=mask)
  return result

def filter_image(image, mask):
    r = image[:,:,0] * mask
    g = image[:,:,1] * mask
    b = image[:,:,2] * mask
    return np.dstack([r,g,b])
  
def otsu_segmentation(img):
    img_gray=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    thresh = threshold_otsu(img_gray)
    img_otsu  = img_gray < thresh
    filtered = filter_image(img, img_otsu)
    return filtered

In [4]:
SIZE = 224
train_image = []
train_label = []
for dir_path in glob.glob("/content/drive/My Drive/train_cro/*"):
    label = dir_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(dir_path,"*.jpg")):
        print(img_path)
        img = cv2.imread(img_path,cv2.IMREAD_COLOR)
        img = cv2.resize(img,(SIZE,SIZE))
        img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        #img = otsu_segmentation(img)
        train_image.append(img)
        train_label.append(label)
train_image = np.array(train_image)
train_label = np.array(train_label)

val_image = []
val_label = []
for dir_path_v in glob.glob("/content/drive/My Drive/validation_cro/*"):
    label_v = dir_path_v.split("\\")[-1]
    #print(label_v)
    for img_path_v in glob.glob(os.path.join(dir_path_v,"*.jpg")):
        #print(img_path_v)
        img = cv2.imread(img_path_v,cv2.IMREAD_COLOR)
        img = cv2.resize(img,(SIZE,SIZE))
        img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        #img = otsu_segmentation(img)
        val_image.append(img)
        val_label.append(label_v)
val_image = np.array(val_image)
val_label = np.array(val_label)

/content/drive/My Drive/train_cro/rust_cro
/content/drive/My Drive/train_cro/rust_cro/100.jpg
/content/drive/My Drive/train_cro/rust_cro/100_1.jpg
/content/drive/My Drive/train_cro/rust_cro/102.jpg
/content/drive/My Drive/train_cro/rust_cro/102_1.jpg
/content/drive/My Drive/train_cro/rust_cro/103.jpg
/content/drive/My Drive/train_cro/rust_cro/103_1.jpg
/content/drive/My Drive/train_cro/rust_cro/104.jpg
/content/drive/My Drive/train_cro/rust_cro/107.jpg
/content/drive/My Drive/train_cro/rust_cro/107_1.jpg
/content/drive/My Drive/train_cro/rust_cro/108.jpg
/content/drive/My Drive/train_cro/rust_cro/109.jpg
/content/drive/My Drive/train_cro/rust_cro/109_1.jpg
/content/drive/My Drive/train_cro/rust_cro/110.jpg
/content/drive/My Drive/train_cro/rust_cro/110_1.jpg
/content/drive/My Drive/train_cro/rust_cro/111.jpg
/content/drive/My Drive/train_cro/rust_cro/112.jpg
/content/drive/My Drive/train_cro/rust_cro/112_1.jpg
/content/drive/My Drive/train_cro/rust_cro/113.jpg
/content/drive/My Drive/t

In [ ]:
from skimage import io
img = train_image[123]
print(img.shape)
#img = cv2.imread(val_image[0])
io.imshow(img)
plt.show()

In [5]:
le = preprocessing.LabelEncoder()
le.fit(train_label)
train_labe_encoded = le.transform(train_label)
le.fit(val_label)
val_label_encoded = le.transform(val_label)

In [6]:
x_train,y_train,x_test,y_test = train_image,train_labe_encoded,val_image,val_label_encoded

In [7]:
x_train,x_test = x_train/255.0,x_test/255.0

In [8]:
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [8]:
model = densenet.DenseNet121(weights=None,input_shape=(224,224,3))
#model.summary()

In [9]:
out = Flatten()(model.get_layer(index = -5).output)

In [10]:
e_model = Model(inputs = model.input,outputs = out)

In [11]:
print("Feature extraction ...")
prediction = np.array(e_model.predict(x_train))
prediction.shape

Feature extraction ...


(2523, 50176)

In [12]:
Xtrain = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

In [13]:
prediction = np.array(e_model.predict(x_test))

In [14]:
Xtest = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

In [15]:
print('\tFeatures training shape: ', Xtrain.shape)
print('\tFeatures testing shape: ', Xtest.shape)

	Features training shape:  (2523, 50176)
	Features testing shape:  (880, 50176)


In [ ]:
print("Classification with Linear SVM ...")
svm = SVC(kernel='linear')
svm.fit(Xtrain, np.ravel(y_train, order='C'))
result = svm.predict(Xtest)

acc = accuracy_score(result, np.ravel(y_test, order='C'))
print("\tAccuracy Linear SVM: %0.4f" % acc)

Classification with Linear SVM ...


In [18]:
print(confusion_matrix(y_test,result))

[[235   6  11]
 [ 13 123  36]
 [ 64  34 166]]


In [19]:
print(classification_report(y_test,result))

              precision    recall  f1-score   support

           0       0.75      0.93      0.83       252
           1       0.75      0.72      0.73       172
           2       0.78      0.63      0.70       264

    accuracy                           0.76       688
   macro avg       0.76      0.76      0.75       688
weighted avg       0.76      0.76      0.76       688



In [39]:
rf = RandomForestClassifier(n_estimators=10,random_state=42)
rf.fit(Xtrain, np.ravel(y_train, order='C'))

RandomForestClassifier(n_estimators=10, random_state=42)

In [40]:
result_rf = rf.predict(Xtest)


In [41]:
acc = accuracy_score(result_rf, np.ravel(y_test, order='C'))
print("\tAccuracy Linear SVM: %0.4f" % acc)
print(confusion_matrix(y_test,result))
print(classification_report(y_test,result))

	Accuracy Linear SVM: 0.7151
[[235   6  11]
 [ 13 123  36]
 [ 64  34 166]]
              precision    recall  f1-score   support

           0       0.75      0.93      0.83       252
           1       0.75      0.72      0.73       172
           2       0.78      0.63      0.70       264

    accuracy                           0.76       688
   macro avg       0.76      0.76      0.75       688
weighted avg       0.76      0.76      0.76       688

